In [2]:
import os, sys, time, subprocess
import numpy as np
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:

# set locations for working files
if len(sys.argv) != 3:
    print("Usage: python3 multi-sim.py <automation_dir> <simulators>")
    print('Assuming testing directories')
    automation_dir = "/mnt/analysis/e17023/Adam/GADGET2/"
    num_sims = 4
else:
    # Automation directory
    automation_dir = sys.argv[1]
    
    # Number of simulators
    num_sims = sys.argv[2]
    try:
        num_sims = int(num_sims)
    except ValueError:
        print("Error: <simulators> must be an integer")
        sys.exit(1)
    if num_sims < 1:
        print("Error: <simulators> must be positive")
        sys.exit(1)
    if num_sims > 10:
        print("Error: <simulators> must be less than 10")
        # highest tested value so far, deminishing returns?
        # probably should talk to IT before increasing this

In [4]:
def start_sim(sim_dir, main_dir):
    if not os.path.isdir(sim_dir):
        os.system("mkdir " + sim_dir)
        print(f"{time.strftime('%H:%M:%S')} | Creating directory {sim_dir}")
    else:
        # delete old output files
        os.system("rm -rf " + sim_dir + "out/*")
        os.system("rm -rf " + sim_dir + "*.tmp") # delete temporary files
        os.system("rm -rf " + sim_dir + "*.log") # delete log files

    
    # copy template files
    os.system("cp " + main_dir + "/simInput/templates/* " + sim_dir)
    
    # convert queue and process notebooks to scripts
    os.system("python3 " + main_dir + "/simInput/nb2py.py " + main_dir + "simInput/multi/queue.ipynb")
    os.system("mv " + main_dir + "simInput/multi/queue.py " + sim_dir)
    os.system("python3 " + main_dir + "/simInput/nb2py.py " + main_dir + "simInput/multi/process.ipynb")
    os.system("mv " + main_dir + "simInput/multi/process.py " + sim_dir)
    os.system("cp " + main_dir + "simInput/multi/sim.sh " + sim_dir)
    os.system("cp " + main_dir + "simInput/multi/param.csv " + sim_dir)
    
    # start simulation in background and return
    os.chdir(sim_dir)
    os.system("chmod +x sim.sh")
    # start simulation, nohup to keep running after logout and & to run in background, supress output
    os.system(f"nohup sim.sh > {sim_dir}log.log 2>&1 &")
    os.chdir(main_dir)
    return None

In [3]:
for simi in range(num_sims): # start all simulations in parallel
    start_sim(f'{automation_dir}Sims/{simi}/', automation_dir)
    print(f"{time.strftime('%H:%M:%S')} | sim{simi} started")
time.sleep(5) # give time for simulations to start

In [ ]:
while True: # simulation management loop    
    statuses = [] # list of statuses for each simulation
    for simi in range(num_sims):        
        param_df = pd.read_csv(automation_dir + "simInput/parameters.csv") # read any updates to parameters.csv
        
        sim_dir = f'{automation_dir}Sims/{simi}/' # directory for current simulation
        sparam = pd.read_csv(sim_dir + "param.csv") # read individual parameter file for current simulation
        
        # check each simulation for status files
        tmp_files = [0]
        for file in os.listdir(sim_dir):
            if file.endswith(".tmp"):
                tmp_files.append(file)
        if tmp_files[-1] == 0:
            tmp_files = ['STOP.tmp']
        statuses.append(tmp_files[-1])
        
        # if simulation is waiting, process output files and queue next run
        if statuses[-1] == 'WAIT.tmp':
            completed_sims = sparam[sparam['Status'] >= 2] # get list of completed sims
            for i in range(len(completed_sims)): # process each completed simulation
                sim_name = completed_sims.iloc[i]['Sim']
                # move sim's output files to main output directory
                if os.path.isfile(f"{sim_dir}out/hdf5/{sim_name}.h5"):
                    os.system(f"mv -f {sim_dir}out/hdf5/{sim_name}.h5 {automation_dir}simOutput/hdf5/")
                if len(os.listdir(f"{sim_dir}out/images/")) > 0:
                    os.system(f"mv -f {sim_dir}out/images/{sim_name}_* {automation_dir}simOutput/images/")
                if len(os.listdir(f"{sim_dir}out/gifs/")) > 0:
                    os.system(f"mv -f {sim_dir}out/gifs/{sim_name}.gif {automation_dir}simOutput/gifs/")
                
                print(f"{time.strftime('%H:%M:%S')} | sim{simi} completed {sim_name} with status {completed_sims.iloc[i]['Status']}")
                param_df.loc[param_df['Sim'] == sim_name, 'Status'] = sparam.loc[sparam['Sim'] == sim_name, 'Status'].values[0]
                param_df.to_csv(automation_dir + "simInput/parameters.csv", index = False)
            
            # queue next run if there are more sims to run
            if len(param_df[param_df['Status'] == 0]) > 0:
                next_params = param_df[param_df['Status'] == 0].head(1) # get parameters for next sim
                next_params.to_csv(sim_dir + "param.csv", index = False) # write parameters to sim's param.csv
                param_df.loc[param_df['Sim'] == next_params['Sim'].iloc[0], 'Status'] = 1 # update status in parameters.csv
                print(f"{time.strftime('%H:%M:%S')} | sim{simi} running {next_params['Sim'].iloc[0]}") # print queued sim number
            else: # no more sims to queue, stop simulation iteration
                with open(sim_dir + "STOP.tmp", "w") as f:
                    f.write(f"{time.strftime('%H:%M:%S')} stopped from manager")
                print(f"{time.strftime('%H:%M:%S')} | sim{simi} stopped")
        param_df.to_csv(automation_dir + "simInput/parameters.csv", index = False) # update parameters.csv with any changes

    status_df = pd.DataFrame({'Sim' : [simi for simi in range(num_sims)], 'Status' : [i.split('.')[0] for i in statuses]})
    status_df.to_csv(automation_dir + "simInput/status.csv", index = False)
    
    if all([status == 'STOP.tmp' for status in statuses]):
        break
    time.sleep(1)